In [1]:
# things we need for NLP
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

# things we need for Tensorflow
import numpy as np
import tflearn
import tensorflow as tf
import random
import re
import unicodedata
import sys

curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [2]:
# restore all of our data structures
import pickle
data = pickle.load( open( "training_data", "rb" ) )
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']
classes_details = data['classes_details']

# import obo file data
import pronto
ont = pronto.Ontology('https://raw.githubusercontent.com/DiseaseOntology/HumanDiseaseOntology/master/src/ontology/doid.obo')

In [3]:
import urllib.request
import urllib.response
import sys
import os, glob
import http.client, urllib
import json
import http.client, urllib.request, urllib.parse, urllib.error, base64

# Replace the accessKey string value with your valid access key.
accessKey = '82e4612615634c398bfd26e7d6327833'
url = 'westcentralus.api.cognitive.microsoft.com'
path = '/text/analytics/v2.0/keyPhrases'

def extract_keywords(body):
    headers = {'Ocp-Apim-Subscription-Key': accessKey}
    conn = http.client.HTTPSConnection(url)
    body_req = {'documents':[{'language': 'en', 'id': 1, 'text': body}]}
    body_json = json.dumps(body_req)
    conn.request ("POST", path, body_json, headers)
    response = conn.getresponse ()
    string = response.read().decode('utf-8')
    json_obj = json.loads(string)
    print("json_obj: ", json_obj)
    keyphrases_list = ((json_obj['documents'])[0])['keyPhrases']
    print("keyphrases_list: ", keyphrases_list)
    return keyphrases_list

In [4]:
# reset underlying graph data
tf.reset_default_graph()
# Build neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

# Define model and setup tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [5]:
## a table structure to hold the different punctuation used

## method to remove punctuations from sentences.
# def remove_punctuation(text):
#     return text.translate(tbl)

def clean_up_sentence(sentence):
    #strip punctuations, numbers, and words containing numbers
    no_nums1 = ' '.join(s for s in sentence.split() if not any(c.isdigit() for c in s))
    no_nums2 = re.sub(r'\d+', '', no_nums1)
    keywords = extract_keywords(no_nums2)
    keyword_str = ' '.join(s for s in keywords)
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(keyword_str)
    # stem each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    print("sentence after stemming: ", sentence_words)
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if s in w.strip().split(): 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

In [6]:
p = bow("i have a fever, arthralgia, and a rash on my arm", words)
print (p)
print (classes)

json_obj:  {'documents': [{'keyPhrases': ['rash', 'fever', 'arthralgia'], 'id': '1'}], 'errors': []}
keyphrases_list:  ['rash', 'fever', 'arthralgia']
sentence after stemming:  ['rash', 'fev', 'arthralg']
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

In [10]:
# load our saved model
model.load('./model.tflearn')

INFO:tensorflow:Restoring parameters from C:\Users\Steven\Documents\HTN Project\model.tflearn


InvalidArgumentError: Restoring from checkpoint failed. This is most likely due to a mismatch between the current graph and the graph from the checkpoint. Please ensure that you have not altered the graph expected based on the checkpoint. Original error:

Assign requires shapes of both tensors to match. lhs shape= [537,8] rhs shape= [590,8]
	 [[Node: save_1/Assign_5 = Assign[T=DT_FLOAT, _class=["loc:@FullyConnected/W"], use_locking=true, validate_shape=true, _device="/job:localhost/replica:0/task:0/device:CPU:0"](FullyConnected/W/Adam, save_1/RestoreV2:5)]]

Caused by op 'save_1/Assign_5', defined at:
  File "c:\users\steven\appdata\local\programs\python\python35\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "c:\users\steven\appdata\local\programs\python\python35\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "c:\users\steven\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py", line 16, in <module>


In [ ]:
ERROR_THRESHOLD = 0.8
def classify(sentence):
    # generate probabilities from the model
    results = model.predict([bow(sentence, words)])[0]
    #print(results)
    # filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    # return tuple of intent and probability
    return return_list

def more_response(key):
    if key in ["yes", "of course", "sure", "certainly", "absolutely", "yeah", "yep", "yup", "ya", "yea", "aye"]:
        sentence_new = input("Okay, please tell me any possible symptoms. The more details, the better!")
        return response(sentence_new,0)
    elif key in ["negative", "nope", "no", "nah", "no way", "nay"]:
        return print("Okay, thanks for visiting iDoctor. Enjoy the rest of your day!")
    else:
        more = input("Sorry, I couldn't understand that. Would you like more diagnosis?")
        return more_response(more.strip().lower())

def response(sentence, counter):
    if counter <= 2:
        results = classify(sentence)
        print("YO RESULTS: ")
        print(results)
    # if we have a classification then find the matching intent tag
        if results and len(sentence) >= 10:
            # loop as long as there are matches to process
            print("Here are some likely conditions you may have:\n")
            for i in range(0,min(3,len(results))):
                print((results[i])[0] + ": " + classes_details[(results[i])[0]])
            print("\nPlease read more into these conditions for possible solutions.\n")
            more = input("Is there anything else you would like me to diagnose?")
            return more_response(more.strip().lower())
        else:
            counter += 1
            sentence_add = input("Unfortunately, I do not have enough information. Please tell me more!")
            sentence = sentence + " " + sentence_add
            print("SENTENCE UP UNTIL THIS POINT: " + sentence)
            return response(sentence, counter)
    else:
        total_responses = ""
        return print("Unfortunately, I cannot help diagnose your medical condition. Please consult a medical professional such as your family doctor. Enjoy the rest of your day!")

def main():
    print("Hi there, my name is iDoctor. Today I will be helping to diagnose any possible medical conditions you may have. Let's get started!")
    name = input("Firstly, what is your name?")
    sentence = input("Hi " + name + "! Please tell me how you are feeling, and any possible symptoms you may be experiencing. The more details, the better!")
    response(sentence, 0)

In [ ]:
classify("i have a fever, arthralgia, and a rash on my arm")

In [ ]:
response("i have a fever, arthralgia, and a rash on my arm")

In [ ]:
main()